In [16]:
import gzip
import json
import numpy as np
import pandas as pd

In [2]:
data = []
with gzip.open("../dataset0.json.gz", "rt") as f:
    for line in f:
        data.append(json.loads(line))

In [8]:
df_labels = pd.read_csv("../data.info.labelled")
label_dict = {
    (row.transcript_id, row.transcript_position): row.label
    for row in df_labels.itertuples(index=False)
}

In [3]:
def get_id_and_position(data):
    id = list(data.keys())[0]
    position = list(data[id].keys())[0]
    return id, position

def get_features(data):
    id, position = get_id_and_position(data)
    return list(data[id][position].values())[0]

In [5]:
get_features(data[0])

[[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0.0093, 10.9, 84.1],
 [0.00631, 2.53, 125.0, 0.00844, 4.67, 126.0, 0.0103, 6.3, 80.9],
 [0.00465, 3.92, 109.0, 0.0136, 12.0, 124.0, 0.00498, 2.13, 79.6],
 [0.00398, 2.06, 125.0, 0.0083, 5.01, 130.0, 0.00498, 3.78, 80.4],
 [0.00664, 2.92, 120.0, 0.00266, 3.94, 129.0, 0.013, 7.15, 82.2],
 [0.0103, 3.83, 123.0, 0.00598, 6.45, 126.0, 0.0153, 1.09, 74.8],
 [0.00398, 3.75, 126.0, 0.00332, 4.3, 129.0, 0.00299, 1.93, 81.9],
 [0.00498, 3.93, 127.0, 0.00398, 2.51, 131.0, 0.0111, 3.47, 79.4],
 [0.0139, 4.69, 106.0, 0.0136, 6.21, 124.0, 0.00531, 10.6, 85.5],
 [0.00631, 3.5, 126.0, 0.0222, 5.38, 128.0, 0.00332, 1.72, 79.3],
 [0.0061, 3.99, 121.0, 0.0121, 7.27, 122.0, 0.00232, 1.27, 78.9],
 [0.00299, 1.99, 128.0, 0.00427, 4.85, 124.0, 0.00332, 3.18, 80.5],
 [0.0186, 3.62, 124.0, 0.00428, 2.25, 129.0, 0.00554, 2.78, 80.1],
 [0.0093, 3.12, 125.0, 0.00398, 8.84, 129.0, 0.00361, 1.86, 82.0],
 [0.00365, 2.92, 126.0, 0.00698, 3.7, 126.0, 0.00467, 3.23, 80.2],
 

In [4]:
feature_length = []
for i in data:
    feature_length.append(len(get_features(i)))

print(np.mean(feature_length))
print(f"minimum number of reads is {np.min(feature_length)}")

90.506295244505
minimum number of reads is 20


Because the minimum number of reads in the data is 20, we try to get the first 20 reads so we have a constant input data of a matrix of 20*9 

In [6]:
def get_top_reads(data, n_reads=20):
    """
    Extracts the first n_reads from the list of reads in `data`.
    Pads with zeros if there are fewer than n_reads.

    Args:
        data (list[list[float]]): Output from get_features(), list of read feature vectors (len = num_reads).
        n_reads (int): Number of reads to keep. Default is 20.

    Returns:
        np.ndarray: Array of shape (n_reads, 9)
    """
    # convert to numpy array
    features = np.array(data, dtype=float)

    # case 1: more reads than needed → take the first n_reads
    if len(features) >= n_reads:
        return features[:n_reads]

    # case 2: fewer reads → pad with zeros
    else:
        pad_len = n_reads - len(features)
        pad = np.zeros((pad_len, features.shape[1]))
        return np.vstack((features, pad))

In [7]:
reads = get_features(data[0])
fixed = get_top_reads(reads, n_reads=20)
print(fixed.shape)

(20, 9)


install torch if you dont have torch on your local environment

In [10]:
 pip install torch torchvision torchaudio

  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 8.0 MB/s  0:00:09m0:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 5.8 MB/s  0:00:01 eta 0:00:01
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [torchaudio]8 [torchaudio]]
Note: you may need to restart the kernel to use updated packages.


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [19]:
! pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 9.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [20]:
from sklearn.model_selection import train_test_split

X = []
y = []

for entry in data:
    id, position = get_id_and_position(entry)
    reads = get_features(entry)
    fixed = get_top_reads(reads, n_reads=20)
    label = label_dict.get((id, int(position)), None)
    X.append(fixed)
    y.append(label)

X = np.array(X)
y = np.array(y)

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

In [21]:
# convert to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_val_t   = torch.tensor(X_val, dtype=torch.float32)
y_val_t   = torch.tensor(y_val, dtype=torch.float32)

train_ds = TensorDataset(X_train_t, y_train_t)
val_ds   = TensorDataset(X_val_t, y_val_t)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False)

In [48]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=9, out_channels=32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 10, 64)  # after pooling 20 -> 10
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        # x shape: (batch, 20, 9)
        x = x.permute(0, 2, 1)  # (batch, features, reads)
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = x.flatten(start_dim=1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x.squeeze(1)

In [49]:
model = SimpleCNN()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
pos_weight = torch.tensor(
    (len(y_train) - y_train.sum()) / y_train.sum(),
    dtype=torch.float32  # force float32 for MPS
).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

only execute the next section if you are on Apple silicon (M1-4 chips), so it will run on GPU via Metal Performance Shaders (MPS)

In [50]:
X_train_t = X_train_t.to(device)
y_train_t = y_train_t.to(device)
X_val_t   = X_val_t.to(device)
y_val_t   = y_val_t.to(device)

In [51]:
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    
    for Xb, yb in train_loader:
        # move batch to device
        Xb, yb = Xb.to(device), yb.to(device)
        
        optimizer.zero_grad()
        y_pred_logits = model(Xb)  # raw logits, do NOT apply sigmoid here
        loss = criterion(y_pred_logits, yb)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # --- validation ---
    model.eval()
    with torch.no_grad():
        y_val_logits = model(X_val_t)  # raw logits
        val_loss = criterion(y_val_logits, y_val_t).item()
        
        # convert logits to probabilities for metrics
        y_val_prob = torch.sigmoid(y_val_logits).detach().cpu().numpy()
        y_val_true = y_val_t.detach().cpu().numpy()
        
        roc_auc = roc_auc_score(y_val_true, y_val_prob)
        pr_auc  = average_precision_score(y_val_true, y_val_prob)
    
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train loss={total_loss/len(train_loader):.4f}, "
          f"Val loss={val_loss:.4f}, "
          f"ROC AUC={roc_auc:.3f}, PR AUC={pr_auc:.3f}")

Epoch 1/30, Train loss=1.3079, Val loss=1.2900, ROC AUC=0.610, PR AUC=0.053
Epoch 2/30, Train loss=1.2942, Val loss=1.2542, ROC AUC=0.649, PR AUC=0.060
Epoch 3/30, Train loss=1.2057, Val loss=1.1584, ROC AUC=0.729, PR AUC=0.179
Epoch 4/30, Train loss=1.0969, Val loss=1.1641, ROC AUC=0.766, PR AUC=0.148
Epoch 5/30, Train loss=1.0667, Val loss=1.0408, ROC AUC=0.796, PR AUC=0.221
Epoch 6/30, Train loss=1.0584, Val loss=1.1866, ROC AUC=0.754, PR AUC=0.146
Epoch 7/30, Train loss=1.0467, Val loss=1.0511, ROC AUC=0.800, PR AUC=0.231
Epoch 8/30, Train loss=1.0466, Val loss=1.0390, ROC AUC=0.799, PR AUC=0.220
Epoch 9/30, Train loss=1.0347, Val loss=1.0211, ROC AUC=0.804, PR AUC=0.249
Epoch 10/30, Train loss=1.0422, Val loss=1.1212, ROC AUC=0.776, PR AUC=0.170
Epoch 11/30, Train loss=1.0376, Val loss=1.0939, ROC AUC=0.798, PR AUC=0.213
Epoch 12/30, Train loss=1.0259, Val loss=1.0248, ROC AUC=0.804, PR AUC=0.251
Epoch 13/30, Train loss=1.0253, Val loss=1.0153, ROC AUC=0.812, PR AUC=0.254
Epoch 14

In [52]:
from sklearn.metrics import roc_auc_score, average_precision_score

## for mac users to copy y_val back to memory
y_val_pred_np = y_val_pred.detach().cpu().numpy()

## if on windows: run 
## y_val_pred_np = y_val_pred.numpy()

roc_auc = roc_auc_score(y_val, y_val_pred_np)
pr_auc = average_precision_score(y_val, y_val_pred_np)
print(f"ROC AUC: {roc_auc:.3f}, PR AUC: {pr_auc:.3f}")

ROC AUC: 0.818, PR AUC: 0.287
